# Unsupervised

This notebook extracts respiration signals, for all subjects in the test dataset, with the following methods:
- Pixel intensity
- Optical flow

In [ ]:
import os
from respiration.dataset import VitalCamSet

dataset = VitalCamSet()
scenarios = dataset.get_scenarios(['101_natural_lighting'])
scenarios

In [ ]:
from datetime import datetime

parameters = {
    'quality_level': 0.1,
    'quality_level_rv': 0.05,
    'use_cgof': True,
}

manifest = {
    'timestamp_start': datetime.now(),
    'scenarios': scenarios,
    'parameters': parameters,
    'incomplete_rois': [],
}

In [ ]:
from tqdm.auto import tqdm
import respiration.roi as roi
import respiration.extractor.pixel_intensity as pixel_intensity
import respiration.extractor.lucas_kanade as lucas_kanade

extracted_signals = []

for (subject, setting) in tqdm(scenarios):
    print(f'Processing {subject} - {setting}')

    frames, params = dataset.get_video_gray(subject, setting)

    areas = roi.get_roi_areas(frames[0])
    if len(areas) < 3:
        print(f'Warning: only {len(areas)} ROIs found for {subject} - {setting}')
        manifest['incomplete_rois'].append({
            'subject': subject,
            'setting': setting,
            'rois': [name for (_, name) in areas],
        })

    for (area, name) in areas:
        #
        # Calculate the average pixel intensity
        #

        pi_start = datetime.now()
        pi_signal = pixel_intensity.average_pixel_intensity(frames, roi=area)
        extracted_signals.append({
            'subject': subject,
            'setting': setting,
            'method': 'pixel_intensity',
            'roi': name,
            'roi_area': list(area),
            'duration': datetime.now() - pi_start,
            'sampling_rate': params.fps,
            'signal': pi_signal.tolist(),
        })

        #
        # Calculate the optical flow
        #

        of_cgof_start = datetime.now()
        of_signal_raw = lucas_kanade.extract_signal(
            frames,
            roi=area,
            quality_level=parameters['quality_level'],
            quality_level_rv=parameters['quality_level_rv'],
            use_cgof=parameters['use_cgof'],
        )

        extracted_signals.append({
            'subject': subject,
            'setting': setting,
            'method': 'lucas_kanade',
            'roi': name,
            'roi_area': list(area),
            'duration': datetime.now() - of_cgof_start,
            'sampling_rate': params.fps,
            'signal': of_signal_raw.tolist(),
        })

    # Garbage collect the frames
    del frames

In [ ]:
manifest['timestamp_finish'] = datetime.now()

In [ ]:
import pandas as pd

df = pd.DataFrame(extracted_signals)
df.head()

In [ ]:
import respiration.utils as utils

output_dir = utils.dir_path('outputs', 'signals', mkdir=True)

# Save the evaluation dataframe
csv_path = os.path.join(output_dir, 'unsupervised_predictions.csv')
df.to_csv(csv_path, index=False)

# Save the hyperparameters as prettified json
json_path = os.path.join(output_dir, 'unsupervised_manifest.json')
utils.write_json(json_path, manifest)